In [ ]:
from utils.binaries import *
from utils.plotting import *
from utils.Auger.SD.Monitoring import Monit
from scipy.signal import argrelextrema
from scipy.optimize import curve_fit
from utils import CONSTANTS
import os, shutil

def parabola(x, scale, mip, y0):
    return scale * (x-mip)**2 + y0

def make_histo_fit(counts, p, mode, t=None, version=None):
     
    os.makedirs(f"/cr/data01/filip/plots/DaqIntegrationTest/{version}/pmt{int(p)}/", exist_ok=True)
    
    # see Framework/SDetector/Station.h
    match p:
        case 0 | 1 | 2:
            ADC = CONSTANTS.UUB_WCD_PEAK if mode == 'peak' else CONSTANTS.UUB_WCD_CHARGE
        case 3:
            ADC = CONSTANTS.UUB_SSD_PEAK if mode == 'peak' else CONSTANTS.UUB_SSD_CHARGE
    
    bin_change = 100 - 1 if mode == 'peak' else 400 - 1
    try:
        __peak = argrelextrema(counts[:bin_change], np.greater, order=bin_change)[0][0]
        x1, x2, y1, y2 = __peak, bin_change, counts[__peak], counts[bin_change]

        slope, offset = -(np.log(y1)-np.log(y2))/(x2-x1), np.log(y1)
        exp_fit = lambda x: np.exp(slope*(x-x1)+offset)

        guess = np.argmax(counts[__peak:bin_change] - exp_fit(range(__peak, bin_change)))
        start, stop = guess - 6, guess + 25
    
        (curve, mip, height), pcov = curve_fit(parabola, ADC[start:stop], counts[start:stop] - exp_fit(range(start, stop)),
                                        bounds=([-np.inf, 0, 0],[0, np.inf, np.inf]),
                                        maxfev=10000,
                                        p0=[-1, guess, counts[guess]])

        file_loc = f"/cr/data01/filip/plots/DaqIntegrationTest/{version}/pmt{int(p)}/{int(t)}-{mode}.png"
        if not os.path.isfile(file_loc):
            fig = plt.figure()
            plt.plot(ADC, counts)
            plt.axvline(ADC[start], c='k', lw=0.3, ls='--')
            plt.axvline(ADC[stop], c='k', lw=0.3, ls='--')
            plt.plot(ADC[start:stop], exp_fit(range(start, stop)), c='k', ls='--', lw=0.5)
            plt.plot(ADC[start:stop], parabola(ADC[start:stop], curve, mip, height) + exp_fit(range(start, stop)))
            plot.save(fig, f'DaqIntegrationTest/{version}/pmt{int(p)}/{int(t)}-{mode}.png')
            plt.close(fig)

        (_, mip, _) = uncertainties.correlated_values([curve, mip, height], pcov)
        if mip.std_dev / mip.n > 0.1: raise ValueError

        # if mip.n > 300 and mode=='peak': raise ValueError

        return mip.n, mip.std_dev
        
    except Exception as e:

        if not os.path.isfile(f"DaqIntegrationTest/{version}/pmt{int(p)}/failed-{int(t)}-{mode}.png"):
            fig = plt.figure()
            plt.plot(ADC, counts)
            plot.save(fig, f"DaqIntegrationTest/{version}/pmt{int(p)}/failed-{int(t)}-{mode}.png")
            plt.close(fig)

        return np.nan, np.nan

In [ ]:
try:
    shutil.rmtree('/cr/data01/filip/plots/daqIntegrationTest/v36', )
except: pass

errorbar_kwargs = [
    {'marker' : 'x', 'c' : 'r'},
    {'marker' : '^', 'c' : 'b'},
    {'marker' : '*', 'c' : 'mediumturquoise'},
    {'marker' : 'o', 'c' : 'k'},
]

version = 'v36'
base = f'/cr/data01/filip/Data/daqIntegrationTests/{version}/'
monit_files = [base + file for file in os.listdir(base) if 'mc' in file]

monit = Monit(*monit_files)
t, (wcd1, wcd2, wcd3) = monit('fTime', 1833 if version >= 'v36' else 136), monit('fPeak', 1833 if version >= 'v36' else 136).T
mask = [True if v != 0 else False for v in wcd3]

_, t_acq, t_gps, pmt, *histos = np.loadtxt(f'/cr/data01/filip/Data/daqIntegrationTests/{version}/peak.txt', unpack=True)
histo_mask = np.where(np.sum(np.array(histos).T[:, :25], axis=1))[0]

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, sharey=True)

try:
    t_muonbuffer, mf_wcd1, hz1, mf_wcd2, hz2, mf_wcd3, hz3, ssd, hz4 = np.loadtxt(f'/cr/data01/filip/Data/daqIntegrationTests/{version}/calib.test', unpack=True)
    t_muonbuffer += CONSTANTS.GPS_OFFSET

    ax2.plot(plot.to_datetime(t_muonbuffer), mf_wcd1, c='r', ls='solid')
    ax2.plot(plot.to_datetime(t_muonbuffer), mf_wcd2, c='blue', ls='solid')
    ax2.plot(plot.to_datetime(t_muonbuffer), mf_wcd3, c='mediumturquoise', ls='solid')
    ax2.plot(plot.to_datetime(t_muonbuffer), ssd, c='k')
except:
    # plot.preliminary(ax2, 'No data =(')
    pass

ax1.plot(plot.to_datetime(t[mask]), wcd1[mask]/0.7, c='r', label='WCD1', ls='solid')
ax1.plot(plot.to_datetime(t[mask]), wcd2[mask]/0.7, c='b', label='WCD2', ls='solid')
ax1.plot(plot.to_datetime(t[mask]), wcd3[mask]/0.7, c='mediumturquoise', label='WCD3', ls='solid')
ax1.plot([],[],c='k', ls='solid', label='SSD')

for i, (tx, p, histo) in enumerate(zip(t_gps[histo_mask], pmt[histo_mask], np.array(histos).T[histo_mask])):

    mip, err = make_histo_fit(histo, p, 'peak', tx, version)
    if p != 3: ax1.errorbar(plot.to_datetime(tx), mip, err, ms=0.4, elinewidth=0.6, **errorbar_kwargs[int(p)])
    # ax2.errorbar(plot.to_datetime(tx), mip, err, ms=0.4, elinewidth=0.6, **errorbar_kwargs[int(p)])

ax1_twin = ax1.twinx()
ax1_twin.legend(title=r'WCD $\times$ 1.42')
ax1_twin.set_ylim(*ax1.get_ylim())
ax1_twin.set_yticklabels([])

ax1.set_ylabel("Xb peak")
ax2.set_ylabel("Muonfill peak")
ax1.legend(bbox_to_anchor=(0, 1.02,1,0.2), loc="lower left", ncol=4)
# ax1.legend(title='Xb')
# ax2.legend(title='MuonFill'